In [13]:
from feature_selector import *

import pandas as pd
import seaborn as sns

from scipy.spatial import distance
from scipy.cluster import hierarchy

import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [14]:
df_ionizable_train: pd.DataFrame = pd.read_csv("../../Data/ionizable_dataset_72_train_divprio.csv", delimiter=';')
df_ionizable_test: pd.DataFrame = pd.read_csv("../../Data/ionizable_dataset_72_test_divprio.csv", delimiter=';')


FS_io_train: FeatureSelector = FeatureSelector(df_ionizable_train)
FS_io_test: FeatureSelector = FeatureSelector(df_ionizable_train)

FS_io_train.scale_data(inplace=True)
FS_io_test.scale_data(inplace=True)


df_neutral_train: pd.DataFrame = pd.read_csv("../../Data/neutral_dataset_111_train_divprio.csv", delimiter=';')
df_neutral_test: pd.DataFrame = pd.read_csv("../../Data/neutral_dataset_111_test_divprio.csv", delimiter=';')

FS_ne_train: FeatureSelector = FeatureSelector(df_neutral_train)
FS_ne_test: FeatureSelector = FeatureSelector(df_neutral_test)

FS_ne_train.scale_data(inplace=True)
FS_ne_test.scale_data(inplace=True)

df_full_train: pd.DataFrame = pd.read_csv("../../Data/full_dataset_train_divprio.csv", delimiter=';')
df_full_test: pd.DataFrame = pd.read_csv("../../Data/full_dataset_test_divprio.csv", delimiter=';')


FS_full_train: FeatureSelector = FeatureSelector(df_full_train)
FS_full_test: FeatureSelector = FeatureSelector(df_full_test)

FS_full_train.scale_data(inplace=True)
print(FS_full_train.df)
FS_full_test.scale_data(inplace=True)


df_all =  [df_ionizable_train, df_ionizable_test, df_neutral_train, df_neutral_test, df_full_train, df_full_test]

         apol       ASA      ASA+      ASA-     ASA_H     ASA_P    a_acc  \
0    0.039739  0.052654  0.051855  0.044585  0.048074  0.048610  0.05698   
1    0.076320  0.075341  0.000000  0.150000  0.000000  0.231884  0.00000   
2    0.053171  0.076801  0.015419  0.134993  0.000000  0.236377  0.05698   
3    0.067034  0.081523  0.109528  0.035056  0.058578  0.112676  0.05698   
4    0.014515  0.030469  0.028113  0.027999  0.026754  0.030642  0.02849   
..        ...       ...       ...       ...       ...       ...      ...   
141  0.064770  0.078362  0.028776  0.122582  0.102203  0.000000  0.00000   
142  0.062131  0.075601  0.031786  0.113588  0.098601  0.000000  0.00000   
143  0.064770  0.077528  0.028777  0.120921  0.101115  0.000000  0.00000   
144  0.059492  0.072341  0.034204  0.104288  0.094350  0.000000  0.00000   
145  0.063372  0.074162  0.021167  0.119882  0.094987  0.004104  0.00000   

     a_acid     a_aro    a_base  ...  vsurf_Wp4  vsurf_Wp5  vsurf_Wp6  \
0       0.0  0

# Low variance features
We try to detect every feature that has a variance below the threshold

## IONIZABLE

In [15]:
print("===== TRAIN =====")
df_ionizable_train_lv, col_ionizable_train_lv = FS_io_train.remove_low_variance(variance_threshold=0)
print("===== TEST =====")
df_ionizable_test_lv, col_ionizable_test_lv = FS_io_test.remove_low_variance(variance_threshold=0)



col_ionizable_lv: set = set(col_ionizable_train_lv) & set(col_ionizable_test_lv)
print(col_ionizable_lv)
print("length of the feature with low variance that are common for train and test: ", len(col_ionizable_lv))



df_ionizable_train = df_ionizable_train.drop(list(col_ionizable_lv), axis=1)
df_ionizable_test = df_ionizable_test.drop(list(col_ionizable_lv), axis=1)

===== TRAIN =====
===== TEST =====
{'MACCS(-40)', 'MACCS(-48)', 'a_nBr', 'MACCS(-14)', 'MACCS(166)', 'a_nP', 'MACCS(--6)', 'MACCS(-46)', 'MACCS(--4)', 'MACCS(--9)', 'MACCS(-21)', 'MACCS(-18)', 'MACCS(-30)', 'MACCS(--2)', 'MACCS(-29)', 'nmol', 'MACCS(-16)', 'MACCS(--1)', 'MACCS(-12)', 'MACCS(-20)', 'MACCS(-49)', 'E_stb', 'MACCS(-27)', 'MACCS(-68)', 'MACCS(-69)', 'MACCS(-26)', 'MACCS(-39)', 'MACCS(-15)', 'MACCS(-64)', 'MACCS(-10)', 'MACCS(-22)', 'MACCS(--5)', 'a_nI', 'vsurf_Wp8', 'MACCS(-31)', 'MACCS(--3)', 'MACCS(-17)', 'MACCS(-44)', 'MACCS(--7)', 'MACCS(-35)', 'a_nB', 'MACCS(-23)'}
length of the feature with low variance that are common for train and test:  42


## NEUTRAL

In [16]:
print("===== TRAIN =====")
df_neutral_train_lv, col_neutral_train_lv =FS_ne_train.remove_low_variance(variance_threshold=0)
print("===== TEST =====")
df_neutral_test_lv, col_neutral_test_lv = FS_ne_test.remove_low_variance(variance_threshold=0)

col_neutral_lv: set = set(col_neutral_train_lv) & set(col_neutral_test_lv)
print(col_neutral_lv)
print("length of the feature that are common for train and test: ", len(col_neutral_lv))

df_neutral_train = df_neutral_train.drop(list(col_neutral_lv), axis=1)
df_neutral_test = df_neutral_test.drop(list(col_neutral_lv), axis=1)

===== TRAIN =====
===== TEST =====
{'MACCS(-40)', 'MACCS(-48)', 'MACCS(-14)', 'MACCS(166)', 'a_nP', 'MACCS(--6)', 'MACCS(--4)', 'MACCS(--9)', 'MACCS(-29)', 'MACCS(-18)', 'MACCS(-30)', 'MACCS(--2)', 'MACCS(-13)', 'MACCS(-24)', 'nmol', 'MACCS(-34)', 'MACCS(-49)', 'MACCS(--1)', 'MACCS(-12)', 'MACCS(-20)', 'MACCS(-68)', 'E_stb', 'MACCS(-27)', 'MACCS(-41)', 'MACCS(-39)', 'MACCS(-15)', 'MACCS(-10)', 'MACCS(--5)', 'a_nI', 'MACCS(-31)', 'MACCS(--3)', 'MACCS(-44)', 'MACCS(--7)', 'MACCS(-35)', 'FCharge', 'a_nB'}
length of the feature that are common for train and test:  36


## FULL

In [17]:
print("===== TRAIN =====")
df_full_train_lv, col_full_train_lv = FS_full_train.remove_low_variance(variance_threshold=0)
print("===== TEST =====")
df_full_test_lv, col_full_test_lv = FS_full_test.remove_low_variance(variance_threshold=0)

col_full_lv: set = set(col_full_train_lv) & set(col_full_test_lv)
print(col_full_lv)
print("length of the feature that are common for train and test: ", len(col_full_lv))

df_full_train = df_full_train.drop(list(col_full_lv), axis=1)
df_full_test = df_full_test.drop(list(col_full_lv), axis=1)

===== TRAIN =====
===== TEST =====
{'MACCS(-40)', 'MACCS(-48)', 'MACCS(-14)', 'MACCS(166)', 'a_nP', 'MACCS(--6)', 'MACCS(--4)', 'MACCS(--9)', 'MACCS(-29)', 'MACCS(-18)', 'MACCS(--2)', 'MACCS(-30)', 'nmol', 'MACCS(--1)', 'MACCS(-12)', 'MACCS(-20)', 'MACCS(-68)', 'E_stb', 'MACCS(-27)', 'MACCS(-39)', 'MACCS(-15)', 'MACCS(-10)', 'MACCS(--5)', 'a_nI', 'MACCS(-31)', 'MACCS(--3)', 'MACCS(-44)', 'MACCS(--7)', 'MACCS(-35)', 'a_nB'}
length of the feature that are common for train and test:  30


# High correlation feature

## IONIZABLE

In [18]:
df_correlation: pd.DataFrame = FS_io_train.get_correlation(df_ionizable_train_lv)

FS_io_train.remove_highly_correlated(df_correlation, df_ionizable_train_lv, threshold=0.8, verbose=True)

vrai df:  (58, 437)
correlation df:  (436, 436)
col:  a_base  |  row:  a_acid  =  0.85
col:  a_count  |  row:  apol  =  0.87
col:  a_donacc  |  row:  a_acc  =  0.9
col:  a_IC  |  row:  ASA  =  0.83
col:  a_IC  |  row:  a_heavy  =  0.82
col:  a_nC  |  row:  apol  =  0.82
col:  BCUT_SMR_0  |  row:  BCUT_PEOE_0  =  0.85
col:  BCUT_SMR_3  |  row:  BCUT_PEOE_3  =  0.83
col:  b_ar  |  row:  a_aro  =  1.0
col:  b_count  |  row:  apol  =  0.9
col:  b_count  |  row:  a_count  =  0.97
col:  b_heavy  |  row:  a_heavy  =  0.95
col:  b_rotN  |  row:  b_1rotN  =  0.93
col:  b_rotR  |  row:  b_1rotR  =  0.88
col:  b_single  |  row:  bpol  =  0.82
col:  chi0  |  row:  a_heavy  =  0.95
col:  chi0  |  row:  a_IC  =  0.83
col:  chi0  |  row:  b_heavy  =  0.87
col:  chi0v  |  row:  apol  =  0.85
col:  chi0v  |  row:  a_heavy  =  0.88
col:  chi0v  |  row:  b_heavy  =  0.85
col:  chi0v  |  row:  chi0  =  0.85
col:  chi0v_C  |  row:  apol  =  0.82
col:  chi0v_C  |  row:  a_nC  =  0.89
col:  chi0v_C  |  row: 

,apol,ASA,ASA+,ASA-,ASA_H,ASA_P,a_acc,a_acid,a_aro,a_don,...,vsurf_IW6,vsurf_IW7,vsurf_IW8,vsurf_R,vsurf_W2,vsurf_Wp2,vsurf_Wp3,vsurf_Wp6,vsurf_Wp7,Log_MP_RATIO
0,0.067094,0.082013,0.087160,0.068147,0.084102,0.063386,0.000000,0.000000,0.082215,0.000000,...,0.000000,0.000000,0.000000,0.121549,0.056140,0.102214,0.128988,0.056833,0.000000,0.711807
1,0.065942,0.097011,0.063272,0.150946,0.041913,0.209648,0.045129,0.325396,0.068512,0.000000,...,0.072004,0.124915,0.125457,0.118383,0.152559,0.074478,0.031784,0.000000,0.000000,0.793092
2,0.065589,0.084661,0.083660,0.081498,0.075775,0.091265,0.000000,0.216930,0.000000,0.000000,...,0.136067,0.163178,0.167556,0.117926,0.075414,0.038013,0.003848,0.000000,0.000000,-1.301030
3,0.046116,0.069428,0.046616,0.105671,0.052776,0.096749,0.045129,0.216930,0.082215,0.094072,...,0.131806,0.164890,0.169943,0.114090,0.092931,0.053765,0.028933,0.045466,0.081111,-1.301030
4,0.136431,0.134472,0.139474,0.117806,0.135614,0.109271,0.090259,0.000000,0.164430,0.000000,...,0.270918,0.000000,0.000000,0.135127,0.132457,0.137013,0.105756,0.045466,0.000000,0.255273
5,0.100534,0.133274,0.042773,0.285341,0.030874,0.350489,0.090259,0.216930,0.000000,0.000000,...,0.191305,0.202618,0.210887,0.130875,0.205561,0.047695,0.006556,0.000000,0.000000,0.367977
6,0.117230,0.128592,0.135843,0.108295,0.133346,0.095927,0.135388,0.000000,0.082215,0.094072,...,0.081728,0.090150,0.119995,0.127692,0.080006,0.114753,0.111885,0.085249,0.000000,-0.431798
7,0.137615,0.145404,0.099051,0.196961,0.102352,0.221754,0.180517,0.216930,0.082215,0.282216,...,0.101333,0.123992,0.134044,0.130195,0.202148,0.150543,0.173599,0.318264,0.243332,-0.202040
8,0.101199,0.109412,0.090271,0.136842,0.116373,0.074795,0.045129,0.000000,0.082215,0.000000,...,0.153875,0.162118,0.167694,0.129088,0.074435,0.095746,0.082096,0.051150,0.000000,0.743902
9,0.080643,0.095416,0.084910,0.108412,0.075290,0.126510,0.045129,0.216930,0.000000,0.000000,...,0.143459,0.141401,0.140221,0.124646,0.114363,0.055313,0.023660,0.000000,0.000000,-1.508638


In [19]:
df_full_train_lv.loc[:,"vsurf_Wp8"]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
141    0.0
142    0.0
143    0.0
144    0.0
145    0.0
Name: vsurf_Wp8, Length: 146, dtype: float64

## Cluster analysis

In [20]:
import pandas as pd
from feature_selector import *

d = {"A": [1,2,3,4], "B": [2,5,6,7], "C": [3,6,8,9], "D":[4,7,9,10]}
df = pd.DataFrame(data=d, index=["A", "B", "C", "D"])
df

FS = FeatureSelector(df)
FS.remove_highly_correlated(df)

vrai df:  (4, 4)
correlation df:  (4, 4)


,A
A,1
B,2
C,3
D,4
